# Best Scenic Design

#### Author: Obuchi Adikema
#### Date: June 27, 2019

## Introduction

Broadway. 

The flash, the style, the elegance. Some of the most important people in making Broadway plays and musicals the sensations that they are, are scenic designers. 

Every year, a Tony Award is presented for Best Scenic Design in a play or musical.
Since the first Tony Awards in 1947, each year a Tony Award was presented for Best Scenic Design until the categories changed in 2005. From then on, two awards have been presented: Best Scenic Design in a Play and Best Scenic Design in a Musical. 

### Who has been awarded and nominated for the most Tony Awards for Best Scenic Design? 
Who historically, has been the best scenic designer? Or at least who has been awarded and nominated for the most Tony Awards in this category. This analysis will use data from the Internet Broadway Database (IBDB). The database contains information on designers, shows, and awards received from 1947-2004.

For this project, I will be building the following table: 
Designers.csv

## Import Primary Packages

In [99]:
import lxml.html
import pandas as pd
import re
import requests

## To answer my question, I must scrape the data from IBDB.

I need to build a dictionary with all of the information that I want to get from the database, so that I can pass it as a parameter into a request. The request can then go to the given URL and pull the information. 

Let's try to get the names of all of the designers. We have access to a database of all of the desingers, not just the scenic ones. That's fine because we will get rid of all of the extra names when we cross-reference with the list of Tony scenic design winners. 

The request below pulls information for all of the designers that were born between the years 1800 and 2005. 

In [77]:
data = {
    "FuncNo": "Designer",
    "birthstartmonth": "01",
    "birthstartday": "01",
    "birthstartyear": "1800",
    "birthendmonth": "06",
    "birthendday": "06",
    "birthendyear": "2005"
}

r = requests.post("http://www.ibdb.com/cast-staff", data=data)

First, we'll get part of the URLs for each designer's page in the database using an xpath. The request we made returns the raw HTML from the webage that has information on all of the designers. The xpath is a specific location that ensures only information from that part of the HTML is pulled. 

We need to create an xpath that is general enough to pull out the information for every designer, but specific enough to leave extra information behind.  

In [78]:
html_tree = lxml.html.fromstring(r.text)
path = "//div[contains(@class, 'person-info')]/div//a/@href"

designers = html_tree.xpath(path)
print("A Peak Inside: ")
print (designers[:10])

A Peak Inside: 
['/broadway-cast-staff/james-acheson-70969', '/broadway-cast-staff/kevin-adams-25504', '/broadway-cast-staff/adrian-24600', '/broadway-cast-staff/ray-aghayan-24603', '/broadway-cast-staff/christopher-akerlind-25507', '/broadway-cast-staff/theoni-v-aldredge-24606', '/broadway-cast-staff/cris-alexander-69325', '/broadway-cast-staff/ren-allio-24612', '/broadway-cast-staff/ralph-alswang-14031', '/broadway-cast-staff/david-amram-11309']


The names of the designers are hidden in bits of URL, but they still need to be extracted further. Here we can use some string functions to generate a "clean" list of names, free of extra words and numbers. 

In [79]:
def clean_names(list):
  new_list = []
  for element in list:
    step_1 = element.rsplit("/")[2]
    step_2 = ''.join([i for i in step_1 if not i.isdigit()]) 
    step_3 = step_2.replace("-", " ")
    new_list.append(step_3.strip())
  return new_list

designerNames = clean_names(designers)

Let's create a database to store all of these names.

In [92]:
df1 = pd.DataFrame(designerNames, columns =['Designer Name'])
df1.head()

,Designer Name
0,james acheson
1,kevin adams
2,adrian
3,ray aghayan
4,christopher akerlind


Now it would be nice to get all of the plays that each designer has worked on. I have to create new request and a new xpath because the plays of each designer are stored on different webpages. The plays are stored at different locations in the HTML. 

In [81]:
path = "//div[@id='broadway']//a/@href"
plays = []

for designer in designers:
  r2 = requests.get("http://www.ibdb.com" + designer)
  html_tree2 = lxml.html.fromstring(r2.text)
  plays.append(html_tree2.xpath(path))

In [82]:
def clean_plays(list):
    inner_list = []
    outer_list = []
    for designer_plays in list:
        for play in designer_plays:
            playNameAlmost = re.match("/[\w\-\.]+/(.*)", play)
            playNameAlmost2 = re.sub("-", " ", playNameAlmost.group(1))
            playName = re.match("[a-z, " "]*", playNameAlmost2)
            inner_list.append(playName.group().strip())
        outer_list.append(inner_list)
        inner_list = []
    return(outer_list)

playNames = clean_plays(plays)

Let's add the plays that each desinger has done to that database.

In [93]:
df1["Plays"] = playNames
df1.head()

,Designer Name,Plays
0,james acheson,[hamlet]
1,kevin adams,"[the cher show, head over heels, spongebob squ..."
2,adrian,"[camelot, obsession, in bed we cry, slightly s..."
3,ray aghayan,"[lorelei, on the town, applause, eddie fisher ..."
4,christopher akerlind,"[time and the conways, indecent, waitress, the..."


In [84]:
def count_plays(grid):
  count_list = []
  for row in grid:
    count_list.append(len(row))
  return count_list

numberOfPlays = count_plays(plays)


In [85]:
df1["Number of Plays"] = numberOfPlays
df1.head()

,Designer Names,Plays,Number of Plays
0,james acheson,[hamlet],1
1,kevin adams,"[the cher show, head over heels, spongebob squ...",25
2,adrian,"[camelot, obsession, in bed we cry, slightly s...",8
3,ray aghayan,"[lorelei, on the town, applause, eddie fisher ...",6
4,christopher akerlind,"[time and the conways, indecent, waitress, the...",23


Now that we have all of the designers, and the plays that they have contributed to, we can check if those designers have had any Tony nods. 

Just like gathering the names of the designers, I need to build a dictionary with all of the information that I want to get from the Tony database, so that I can pass it as a parameter into a request. The request can then go to the given URL for the awards webapge and pull the information. 

The request below pulls information for all of The Tony Awards for Best Scenic Design pre-2005. 

In [86]:
award_data_2005 = {
    "AwdAliasNo": "1001",
    "Year": "",
    "AwdCatNo": "1017"
}

r3 = requests.post("http://www.ibdb.com/awards", data=award_data_2005) 

In [87]:
path = "//div[contains(string(), 'Winner')]//preceding-sibling::a[contains(@href, 'cast-staff')]/@href"

html_tree3 = lxml.html.fromstring(r3.text)
winnerNomineeNames = clean_names(html_tree3.xpath(path))
print("A Peak Inside: ")
print (winnerNomineeNames[:10])

A Peak Inside: 
['eugene lee', 'robert brill', 'tom pye', 'ralph funicello', 'catherine martin', 'john lee beatty', 'david rockwell', 'santo loquasto', 'tim hatley', 'douglas w schmidt']


## Calculate which designers have won Tonys for Best Scenic Design and how many they have won.

In [88]:
tonyAwardWN = []
numberOfAwards = []
tony = 0
awardCounter = 0

for designer in designerNames: 
    for winner in winnerNomineeNames:
        if designer == winner:
            tony = 1
            awardCounter = awardCounter + 1
    tonyAwardWN.append(tony)
    numberOfAwards.append(awardCounter)
    tony = 0
    awardCounter = 0

1 means that the designer has been nominated for or won a Tony Award for Best Scenic Design. 0 means that the designer has never even been nominated for an award in the category. 

In [94]:
df1["Tony?"] = tonyAwardWN
df1["Number of Tony Nominations/Wins"] = numberOfAwards
df1.head()

,Designer Name,Plays,Tony?,Number of Tony Nominations/Wins
0,james acheson,[hamlet],0,0
1,kevin adams,"[the cher show, head over heels, spongebob squ...",0,0
2,adrian,"[camelot, obsession, in bed we cry, slightly s...",0,0
3,ray aghayan,"[lorelei, on the town, applause, eddie fisher ...",0,0
4,christopher akerlind,"[time and the conways, indecent, waitress, the...",0,0


Now that we have a completed database of information about the broadway designers, we can store everything into a seperate CSV file, in case anyone wants to access that information later.

In [96]:
df1.to_csv("Designers.csv")

## Now, let's finally find out which scenic designer has received the most nods and wins from The Tony Awards. 

In [98]:
greatestWinner = designerNames[numberOfAwards.index(max(numberOfAwards))]
mostAwards = max(numberOfAwards)
totalPlays = numberOfPlays[numberOfAwards.index(max(numberOfAwards))]

announcement = "## The designer with the most Tony Awards for Best Scenic Design is "+ str(greatestWinner).title() + ", with a combination of " + str(mostAwards) + " wins and nominations. They have been credited for work on " + str(totalPlays) + " plays."

display(Markdown(announcement))

## The designer with the most Tony Awards for Best Scenic Design is Oliver Smith, with a combination of 23 wins and nominations. They have been credited for work on 138 plays.

## References

[Internet Broadway Database](http://www.ibdb.com)